## 翻译数据集

In [1]:
import os
with open(os.path.join('cmn.txt'), 'r',
          encoding='utf-8') as f:
    raw_text = f.read()

print(raw_text[:200])

FileNotFoundError: [Errno 2] No such file or directory: 'cmn.txt'

In [ ]:
def preprocess(text):
    """预处理“英语-法语”数据集"""

    def no_space(char, prev_char):
        """
        判断是否在当前字符前添加空格。
        如果当前字符是 `, . ! ?` 之一，并且前一个字符不是空格，则返回 True（需要添加空格）。
        """
        return char in set(',.!?') and prev_char != ' '

    # '\u202f' 是窄不间断空格 (narrow no-break space)
    # '\xa0' 是不间断空格 (no-break space)
    text = text.replace('\u202f', ' ').replace('\xa0', ' ')

    # 将文本转换为小写，标准化文本格式
    text = text.lower()

    # 在单词和标点符号之间插入空格
    out = [
        ' ' + char if i > 0 and no_space(char, text[i - 1]) else char
        for i, char in enumerate(text)  # 遍历文本的每个字符
    ]

    # 将字符列表合并为字符串
    return ''.join(out)

# 示例输入
text = preprocess(raw_text)  # 预处理原始文本
print(text[:80])  # 打印前 80 个字符

In [ ]:
def tokenize(text, num_examples=None):
    """对“英语—法语”翻译数据集进行【单词级】词元化 (Tokenization)"""

    # 初始化存储源语言（英语）和目标语言（法语）句子的列表
    source, target = [], []

    # 遍历文本的每一行，并按 `\n` (换行符) 拆分文本
    for i, line in enumerate(text.split('\n')):

        # 如果指定了 `num_examples`，则只处理最多 `num_examples` 行
        if num_examples and i > num_examples:
            break

        # 按 `\t` (制表符) 拆分行，得到英语句子和对应的法语翻译
        parts = line.split('\t')

        # 只有当该行包含两个部分（即英语和法语）时才进行处理
        if len(parts) == 2:

            # 将英语句子按空格拆分为单词列表，并添加到 `source` (源语言) 列表
            source.append(parts[0].split(' '))

            # 将法语句子按空格拆分为单词列表，并添加到 `target` (目标语言) 列表
            target.append(parts[1].split(' '))

    # 返回处理后的两个列表，分别存储英语和法语的 token 列表
    return source, target

# 词元化整个数据集
# 注意：这里的 text 应该是上面读取到的 raw_text
source, target = tokenize(raw_text, num_examples=30)

# 打印处理后的结果示例
for s, t in zip(source, target):
    print('english:', s, '中文:', t)
    # 如果只想看前几个示例，可以加一个 break 或限制循环次数

In [ ]:
import collections
import re

import math
import time

import torch
from torch import nn
import torch.nn.functional as F

import collections

def count_corpus(tokens):
    """统计词元频率"""
    # 如果 tokens 是二维列表（如 source），将其展平为一维
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)


class Vocab:
    def __init__(self, tokens=None):
        if tokens is None:
            tokens = []

        # 1. 统计词频
        counter = count_corpus(tokens)

        # 2. 初始化特殊符号 (这里的 ' ' 通常作为 <pad> 使用)
        self.idx_to_token = ['<pad>', '<unk>', '<bos>', '<eos>']
        self.token_to_idx = {
            '<pad>': 0,
            '<unk>': 1,
            '<bos>': 2,
            '<eos>': 3,
        }

        # 3. 按频率从高到低加入普通 token
        # counter.most_common() 返回 (token, freq) 的列表
        for token, freq in counter.most_common():
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        # 如果是单个单词，返回其索引；如果没找到，返回 <unk> 的索引
        if isinstance(tokens, str):
            return self.token_to_idx.get(tokens, self.token_to_idx['<unk>'])
        # 如果是单词列表，递归处理
        return [self[token] for token in tokens]

    def to_tokens(self, indices):
        """根据索引转回单词 (方便调试)"""
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    def print_vocab(self, n=10):
        print("===== Vocabulary Preview =====")
        print("index -> token")
        for i in range(min(n, len(self.idx_to_token))):
            print(f"{i:>3} -> {self.idx_to_token[i]}")



# 假设 source 已经通过前面的 tokenize(raw_text) 生成了
# 示例数据: source = [['Go.'], ['Hi.'], ['Run!'], ['Run!']]

# 初始化英语词表
src_vocab = Vocab(source)

# 打印词表预览
src_vocab.print_vocab(15)

# 测试：将一个句子转为索引
example_sentence = source[0] # 比如 ['Go.']
indices = src_vocab[example_sentence]

print(f"\nExample sentence: {example_sentence}")
print(f"Indices: {indices}")
print(f"Back to tokens: {src_vocab.to_tokens(indices)}")

## truancate 和 padding！

In [ ]:
def truncate_pad(line, num_steps, padding_token):
    """截断或填充文本序列"""
    if len(line) > num_steps:
        return line[:num_steps]  # 截断
    return line + [padding_token] * (num_steps - len(line))  # 填充

# 示例逻辑
# truncate_pad(src_vocab[source[0]], 10, src_vocab['<pad>'])

def build_array(lines, vocab, num_steps):
    """将机器翻译的文本序列转换成小批量数据 (batch)"""
    # 1. 将每个单词转换成对应的索引
    lines = [vocab[l] for l in lines]

    # 2. 在每个句子结尾添加 <eos> 标记
    lines = [l + [vocab['<eos>']] for l in lines]

    # 3. 截断或填充，确保长度固定为 num_steps，并转为 Tensor
    array = torch.tensor([truncate_pad(l, num_steps, vocab['<pad>']) for l in lines])

    # 4. 计算有效长度 (排除 <pad> 后的实际单词数)
    valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)

    return array, valid_len

from torch.utils import data

def load_array(data_arrays, batch_size, is_train=True):
    """
    构造一个 PyTorch 数据迭代器。

    参数:
    data_arrays (tuple): 包含 (src_array, src_valid_len, tgt_array, tgt_valid_len) 的元组
    batch_size (int): 每个小批量的大小
    is_train (bool): 是否在迭代时打乱数据
    """
    # 1. 将数据封装成 TensorDataset
    # *data_arrays 会将元组解包，传入对应的张量
    dataset = data.TensorDataset(*data_arrays)

    # 2. 返回 DataLoader 实例
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

# 注意：此函数依赖前面定义的 tokenize, Vocab 类, build_array 和 truncate_pad
def load_data(batch_size, num_steps, num_examples=600):
    """返回翻译数据集的迭代器和词表"""
    with open(os.path.join('cmn.txt'), 'r', encoding='utf-8') as f:
        raw_text = f.read()

    # 假设 preprocess 函数已定义，或者直接使用 raw_text
    # text = preprocess(raw_text)
    source, target = tokenize(raw_text, num_examples)

    # 构建词表
    src_vocab = Vocab(source)
    tgt_vocab = Vocab(target)

    # 生成张量数组
    src_array, src_valid_len = build_array(source, src_vocab, num_steps)
    tgt_array, tgt_valid_len = build_array(target, tgt_vocab, num_steps)

    data_arrays = (src_array, src_valid_len, tgt_array, tgt_valid_len)

    # 假设 load_array 是一个从 Tensor 创建 DataLoader 的工具函数
    # from utils import load_array
    data_iter = load_array(data_arrays, batch_size)

    return data_iter, src_vocab, tgt_vocab

# 运行示例
train_iter, src_vocab, tgt_vocab = load_data(batch_size=2, num_steps=8)


# 1. 获取第一个小批量数据
# X: 源语言索引, X_valid_len: 源语言有效长度
# Y: 目标语言索引, Y_valid_len: 目标语言有效长度
X, X_valid_len, Y, Y_valid_len = next(iter(train_iter))

print(f"当前 Batch 大小: {X.shape[0]}, 句子最大长度 (num_steps): {X.shape[1]}")
print("-" * 50)

# 2. 遍历并打印这个 Batch 里的每一个 Case
for i in range(X.shape[0]):
    # 使用 src_vocab 的 idx_to_token 属性进行转换
    # X[i] 是一个 tensor，tolist() 转为列表以便索引
    src_tokens = [src_vocab.idx_to_token[int(idx)] for idx in X[i]]
    tgt_tokens = [tgt_vocab.idx_to_token[int(idx)] for idx in Y[i]]

    print(f"Case {i+1}:")
    print(f"  [Source]: {' '.join(src_tokens)}")
    print(f"  [Valid L]: {X_valid_len[i].item()}")
    print(f"  [Target]: {' '.join(tgt_tokens)}")
    print(f"  [Valid L]: {Y_valid_len[i].item()}")
    print("-" * 50)

## Encoder Decoder

In [ ]:
from torch import nn

class Encoder(nn.Module):
    """编码器基类"""
    def __init__(self, **kwargs):
        super(Encoder, self).__init__(**kwargs)

    def forward(self, X, *args):
        raise NotImplementedError

class Decoder(nn.Module):
    """解码器基类"""
    def __init__(self, **kwargs):
        super(Decoder, self).__init__(**kwargs)

    def init_state(self, enc_outputs, *args):
        raise NotImplementedError

    def forward(self, X, state):
        raise NotImplementedError

class EncoderDecoder(nn.Module):
    """编码器-解码器架构的基类"""
    def __init__(self, encoder, decoder, **kwargs):
        super(EncoderDecoder, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)
        return self.decoder(dec_X, dec_state)

In [2]:
class Seq2SeqEncoder(Encoder):
    """用于序列到序列学习的循环神经网络编码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        # 嵌入层：将词索引转换为稠密向量
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # RNN 层：使用多层门控循环单元 (GRU)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers, dropout=dropout)

    def forward(self, X, *args):
        # 输入 X 的形状: (batch_size, num_steps) -> [批量大小, 时间步数]

        # 1. 特征映射：将索引转为词向量
        # 输出形状: (batch_size, num_steps, embed_size)
        X = self.embedding(X)

        # 2. 维度转换：PyTorch 的 RNN 默认期望 (num_steps, batch_size, embed_size)
        # 将“时间步”换到第一维
        X = X.permute(1, 0, 2)

        # 3. 前向传播
        # output 形状: (num_steps, batch_size, num_hiddens) -> 包含所有时刻的隐状态
        # state 形状: (num_layers, batch_size, num_hiddens) -> 包含每一层最后时刻的隐状态
        output, state = self.rnn(X)

        # 返回输出和最终状态，状态将作为解码器的初始状态
        return output, state

class Seq2SeqDecoder(Decoder):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout=0, **kwargs):
        super(Seq2SeqDecoder, self).__init__(**kwargs)
        # 解码器也有自己的嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # RNN 层：注意输入维度是 embed_size + num_hiddens
        # 因为我们要把当前词向量和编码器传来的“上下文向量”拼在一起作为输入
        self.rnn = nn.GRU(embed_size + num_hiddens, num_hiddens, num_layers, dropout=dropout)

        # 输出层：将隐藏状态映射回词表大小的维度，用于预测下一个词
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, *args):
        # 接收编码器的输出，并提取其最终隐藏状态作为初始状态
        # enc_outputs[1] 即编码器的最终 state
        return enc_outputs[1]

    def forward(self, X, state):
        # 输入 X 形状: (batch_size, num_steps)
        # state 形状: (num_layers, batch_size, num_hiddens)

        # 1. 词向量化并调整维度 -> (num_steps, batch_size, embed_size)
        X = self.embedding(X).permute(1, 0, 2)

        # 2. 提取上下文向量：
        # 选取编码器最后一层的最后时间步状态作为“上下文”
        # state[-1] 形状: (batch_size, num_hiddens)
        # 使用 repeat 将其复制，使其在每个时间步都可见 -> (num_steps, batch_size, num_hiddens)
        context = state[-1].repeat(X.shape[0], 1, 1)

        # 3. 特征拼接：将当前输入 X 与上下文 context 在特征轴 (dim=2) 拼接
        # 拼接后形状: (num_steps, batch_size, embed_size + num_hiddens)
        X_and_context = torch.cat((X, context), 2)

        # 4. 前向传播：传入拼接后的向量和上一时刻的状态
        output, state = self.rnn(X_and_context, state)

        # 5. 映射输出：通过全连通层，并把维度调回 (batch_size, num_steps, vocab_size)
        output = self.dense(output).permute(1, 0, 2)

        # 返回预测概率分布和更新后的状态（用于下一个时间步）
        return output, state

NameError: name 'Encoder' is not defined

## mask！

In [ ]:
def sequence_mask(X, valid_len, value=0):
    """
    在序列中屏蔽 (mask) 不相关的项，通常用于对填充部分进行屏蔽。
    参数:
        X (torch.Tensor): 形状为 (batch_size, maxlen) 的输入张量。
        valid_len (torch.Tensor): 形状为 (batch_size,) 的张量，表示每个序列的有效长度。
        value (int or float, 可选): 需要屏蔽的位置填充的数值，默认值为 0。
    返回:
        torch.Tensor: 处理后的张量，其中无效部分被替换为 `value`。
    """
    # 获取输入张量 X 的最大序列长度 (num_steps)，即 X 的第二维度
    maxlen = X.size(1)

    # 生成一个形状为 (1, maxlen) 的张量，其值为 0 到 maxlen-1
    # - `torch.arange(maxlen)` 创建一个 1D 向量 [0, 1, 2, ..., maxlen-1]
    # - `device=X.device` 确保计算在相同设备 (如 GPU/CPU) 上进行
    # - `[None, :]` 使其扩展为形状 (1, maxlen)，以便与 `valid_len[:, None]` 进行广播
    mask = torch.arange(maxlen, dtype=torch.float32, device=X.device)[None, :]

    # 生成一个形状为 (batch_size, maxlen) 的布尔掩码矩阵
    # - `valid_len[:, None]` 将 `valid_len` 变为形状 (batch_size, 1)，以进行广播
    # - `mask < valid_len[:, None]` 逐元素比较：
    #   - 若 `mask` 中的索引小于 `valid_len`，则该位置为 True (有效)
    #   - 否则，该位置为 False (填充部分需要屏蔽)
    mask = mask < valid_len[:, None]

    # 利用掩码对 `X` 进行屏蔽 (将无效部分替换为 `value`)
    # - `~mask` 取反，将 False (填充部分) 变为 True，True (有效部分) 变为 False
    # - `X[~mask] = value` 将 `X` 中无效部分替换为 `value`
    X[~mask] = value

    return X

In [ ]:
class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    """带遮蔽(masking)的 softmax 交叉熵损失函数"""
    def forward(self, pred, label, valid_len):
        """
        参数:
            pred (tensor): 形状 (batch_size, num_steps, vocab_size) -> 预测的概率分布
            label (tensor): 形状 (batch_size, num_steps) -> 真实标签
            valid_len (tensor): 形状 (batch_size,) -> 每个序列的有效长度 (不包括 <pad>)
        """
        # 创建与 `label` 形状相同的 `weights` 张量, 并全部初始化为 1
        weights = torch.ones_like(label)
        # 通过 `sequence_mask()` 生成遮蔽 (mask), 过滤填充部分
        weights = sequence_mask(weights, valid_len)
        # 禁用 CrossEntropyLoss 的默认 `reduction` 选项, 让其计算逐元素损失
        self.reduction = 'none'

        # 计算未加权的交叉熵损失
        # CrossEntropyLoss 期望输入形状为 (batch_size, vocab_size, num_steps), 所以需要 permute
        unweighted_loss = super(MaskedSoftmaxCELoss, self).forward(
            pred.permute(0, 2, 1), label)

        # 计算加权损失: 填充部分的损失变为 0 (不会影响梯度)
        # 对 `num_steps` 维度求均值, 得到每个样本的最终损失
        weighted_loss = (unweighted_loss * weights).mean(dim=1)

        return weighted_loss # **返回每个样本的损失**

In [ ]:
class Accumulator:
    """
    在多个变量上进行累加的工具类
    """
    def __init__(self, n):
        """
        参数:
            n (int): 需要累加的变量个数
        """
        self.data = [0.0] * n

    def add(self, *args):
        """
        将传入的值逐项累加
        """
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        """
        清零
        """
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        """
        允许用 metric[i] 的方式访问
        """
        return self.data[idx]

def train_seq2seq(net, data_iter, lr, num_epochs, tgt_vocab, device):
    """
    训练序列到序列 (Seq2Seq) 模型
    """
    # 1. 权重初始化：使用 Xavier 初始化使模型在训练初期更稳定
    def xavier_init_weights(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
        if type(m) == nn.GRU:
            # GRU 有多个内部参数矩阵，需要遍历初始化
            for param in m._flat_weights_names:
                if "weight" in param:
                    nn.init.xavier_uniform_(m._parameters[param])

    net.apply(xavier_init_weights) # 应用初始化
    net.to(device)                 # 模型搬运到 GPU/CPU

    optimizer = torch.optim.Adam(net.parameters(), lr=lr) # 使用 Adam 优化器
    loss = MaskedSoftmaxCELoss()                          # 使用自定义的带遮蔽损失函数

    net.train() # 设置为训练模式

    for epoch in range(num_epochs):
        # 使用你提供的 Accumulator 类，累加 2 个变量：损失总和、有效词元总数
        metric = Accumulator(2)

        for batch in data_iter:
            optimizer.zero_grad() # 梯度清零

            # X: 源语言, X_valid_len: 源语言有效长度
            # Y: 目标语言, Y_valid_len: 目标语言有效长度
            X, X_valid_len, Y, Y_valid_len = [x.to(device) for x in batch]

            # 2. 构造解码器的输入 (强制教学 Teacher Forcing)
            # 在 Y (目标句子) 前面拼接一个 <bos> 标记
            bos = torch.tensor([tgt_vocab['<bos>']] * Y.shape[0], device=device).reshape(-1, 1)
            # dec_input = <bos> + Y 的前 n-1 个词 (丢弃最后一个词以保持步数一致)
            # 例如 Y 为 [我, 是, 学生, <eos>]，则输入为 [<bos>, 我, 是, 学生]
            dec_input = torch.cat([bos, Y[:, :-1]], 1)

            # 3. 前向传播
            # Y_hat 形状: (batch_size, num_steps, vocab_size)
            Y_hat, _ = net(X, dec_input, X_valid_len)

            # 4. 计算损失
            # l 返回的是每个样本的平均损失
            l = loss(Y_hat, Y, Y_valid_len)
            l.sum().backward() # 损失求和后反向传播计算梯度

            # 5. 梯度裁剪 (Gradient Clipping)
            # 防止 RNN 常见的梯度爆炸问题，将梯度范数限制在 1 以内
            torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1)

            # 6. 更新权重
            num_tokens = Y_valid_len.sum() # 计算当前 batch 中所有非 <pad> 的词数
            optimizer.step()

            # 7. 统计指标
            with torch.no_grad():
                metric.add(l.sum(), num_tokens)

        # 每一轮结束打印一次平均训练损失 (Perplexity 或 Cross Entropy)
        # metric[0] 是总损失，metric[1] 是总词数
        epoch_loss = metric[0] / metric[1]
        print(f'Epoch {epoch + 1}: loss = {epoch_loss:.3f}')

    print("训练结束！")

In [ ]:
embed_size, num_hiddens, num_layers, dropout = 32, 32, 2, 0.1
batch_size, num_steps = 64, 10
lr, num_epochs, device = 0.005, 300, "cpu"

train_iter, src_vocab, tgt_vocab = load_data(batch_size, num_steps)

encoder = Seq2SeqEncoder(
    len(src_vocab),
    embed_size,
    num_hiddens,
    num_layers,
    dropout
)

decoder = Seq2SeqDecoder(
    len(tgt_vocab),
    embed_size,
    num_hiddens,
    num_layers,
    dropout
)

net = EncoderDecoder(encoder, decoder)
train_seq2seq(net, train_iter, lr, num_epochs, tgt_vocab, device)
